In [2]:
import librosa
import os
import glob
from scipy.io import wavfile
from scipy import signal
import numpy as np
import re

In [3]:
# Path to the directory containing folders with .wav files
root_path = "d:\Sistema\Escritorio\Escritorio\Tesis\DAIC-WOZ\data"

# Define the root path for saving progress
PROGRESS_PATH = os.path.join(os.getcwd(), 'progress.txt')

In [4]:
# Function to save progress
def save_progress( wav_file_path):
    with open(PROGRESS_PATH, 'w') as file:
        file.write(wav_file_path)

# Function to load progress
def load_progress():
    progress_file = PROGRESS_PATH
    if os.path.exists(progress_file):
        with open(progress_file, 'r') as file:
            return file.read().strip()
    else:
        return None

In [5]:
def extract_numbers_from_string(input_string):
    # Regular expression to match numbers
    pattern = r'\d+'
    # Find all numbers in the input string
    numbers = re.findall(pattern, input_string)
    return numbers

In [6]:
def standard_normal_variate(data):
    mean = np.mean(data)
    std = np.std(data)

    return (data - mean) / std


def create_mfcc_delta(feature, concat=False):
    """
    Obtains the local differential (first and second order) of the MFCC

    Inputs
        feature: np.array - The MFCC to be used for the local differentials
        concat: bool - If True, the differentials will be concatenated rather
                than stacked

    Output
        mfcc: numpy.array - The Updated MFCC
    """
    mfcc_delta = librosa.feature.delta(feature)
    mfcc_delta2 = librosa.feature.delta(feature, order=2)

    if concat:
        mfcc = np.concatenate((feature, mfcc_delta, mfcc_delta2))
    else:
        mfcc = np.array((feature, mfcc_delta, mfcc_delta2))

    return mfcc


def mfcc(audio, sample_rate, freq_bins, window_size, hop_size,
         window_func=np.hanning(1024), snv=False):
    """
    Obtains the local differential (first and second order) of the MFCC

    Inputs
        audio: np.array - The audio data to be converted to MFCC
        sample_rate: int - The original sampling rate of the audio
        freq_bins: int - The number of mel bins
        window_size: int - The length of the window to be passed over the data
        hop_size: int - The gap between windows
        window: numpy - The type of window function to be used

    Output
        mfcc: numpy.array - The Updated MFCC
    """
    mfcc = librosa.feature.mfcc(y=audio,
                                sr=sample_rate,
                                n_mfcc=freq_bins,
                                n_fft=window_size,
                                hop_length=hop_size,
                                window=window_func)

    if snv:
        mfcc = standard_normal_variate(mfcc)

    return mfcc


In [23]:
# Generate preprocessed files
# Load progress
last_wav_file_path = load_progress()
def processAudio():
  for folder_name in os.listdir(root_path):
    folder_path = os.path.join(root_path, folder_name)
    if os.path.isdir(folder_path):
      if last_wav_file_path!= None and extract_numbers_from_string(folder_path) < extract_numbers_from_string(last_wav_file_path):
        continue
      for wav_file_path in glob.glob(os.path.join(folder_path, 'preprocessed_*.wav')):
        # Save progress
        save_progress(folder_path)
            
        print(folder_path)
        # load data
        rate, data = wavfile.read(wav_file_path)
        data = data.astype(np.float64)
        # perform noise reduction
        mfcc_data = mfcc(data, rate, 8, 512, 512, np.hanning(512))
        print(mfcc_data)
        # Extract the audio file name without extension
        audio_file_name = os.path.splitext(os.path.basename(wav_file_path))[0]
        # new file name with folder route
        file_name = os.path.join(folder_path, f'preprocessed_{audio_file_name}.wav')
      
processAudio()

d:\Sistema\Escritorio\Escritorio\Tesis\DAIC-WOZ\data\492_P
[[ 1.82517275e+02  1.82517275e+02  1.82517275e+02 ...  2.80228330e+02
   4.26012697e+02  3.81955847e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  1.05510359e+00
   1.85420701e+02  1.55209736e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  3.23933443e+00
  -3.27901745e+01 -2.95570615e+01]
 ...
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ... -1.50300899e+00
  -1.67417294e+01 -9.87662127e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ... -8.30818698e+00
  -3.87680774e+01 -4.10484776e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ... -1.57833821e-01
  -2.91816907e+01 -1.53026255e+01]]
